Sarthak Shrivastava

# Transverse Vibration Analysis of Euler-Bernoulli Beam using Finite DIfference Method

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib widget

In [ ]:
# To find natural frequencies and mode shapes of transverse vibration of beam (using FDM)

def transverse_vibration_modal(E, I, A, rho, L, BC1, BC2, n):

    dx = L / (n - 1)

    P = np.zeros((n, n))

    for i in range(2, n - 2):
        P[i][i-2] = 1
        P[i][i-1] = -4
        P[i][i] = 6
        P[i][i+1] = -4 
        P[i][i+2] = 1

    # Boundary conditions at x = 0

    if (BC1 == 1): # Fixed end
        P[0][0] = 0
        P[1][0] = -4
        P[1][1] = 7
        P[1][2] = -4
        P[1][3] = 1

    elif (BC1 == 2): # Pinned end
        P[0][0] = 0
        P[1][0] = -2
        P[1][1] = 5
        P[1][2] = -4
        P[1][3] = 1

    elif (BC1 == 3): # Free end
        P[0][0] = 2
        P[0][1] = -4
        P[0][2] = 2
        P[1][0] = -2
        P[1][1] = 5
        P[1][2] = -4
        P[1][3] = 1

    # Boundary conditions at x = L

    if (BC2 == 1): # Fixed end
        P[n-1][n-1] = 0
        P[n-2][n-1] = -4
        P[n-2][n-2] = 7
        P[n-2][n-3] = -4
        P[n-2][n-4] = 1

    elif (BC2 == 2): # Pinned end
        P[n-1][n-1] = 0
        P[n-2][n-1] = -2
        P[n-2][n-2] = 5
        P[n-2][n-3] = -4
        P[n-2][n-4] = 1

    elif (BC2 == 3): # Free end
        P[n-1][n-1] = 2
        P[n-1][n-2] = -4
        P[n-1][n-3] = 2
        P[n-2][n-1] = -2
        P[n-2][n-2] = 5
        P[n-2][n-3] = -4
        P[n-2][n-4] = 1

    # Solve eigen value problem for natural frequencies and mode shapes
    if ((BC1 == 1 or BC1 == 2) and (BC2 == 1 or BC2 == 2)): # No free end
        P1 = P[1:n-1, 1:n-1]
    elif (BC1 == 3 and (BC2 == 1 or BC2 == 2)): # Node 1 is free
        P1 = P[0:n-1, 0:n-1]
    elif (BC2 == 3 and (BC1 == 1 or BC1 == 2)): # Node n is free
        P1 = P[1:, 1:]

    eig, eigv = np.linalg.eigh(P1)

    # Natural frequencies
    omega = np.sqrt((eig) * (E * I / (rho * A))) / dx**2 

    # Mode Shapes
    phi = eigv # Modal matrix
    if ((BC1 == 1 or BC1 == 2) and (BC2 == 1 or BC2 == 2)): # No free end
        phi = np.append([[0 for i in range(n-2)]], phi, axis = 0)
        phi = np.append(phi, [[0 for i in range(n-2)]], axis = 0)
    elif (BC1 == 3 and (BC2 == 1 or BC2 == 2)): # Node 1 is free and Node n is fixed/pinned
        phi = np.append(phi, [[0 for i in range(n-1)]], axis = 0)
    elif (BC2 == 3 and (BC1 == 1 or BC1 == 2)): # Node n is free and Node 1 is fixed/pinned
        phi = np.append([[0 for i in range(n-1)]], phi, axis = 0)

    return omega, phi

In [ ]:
# To find transverse vibration response of beam (using FDM)

def transverse_vibration_response(E, I, A, rho, L, BC1, BC2, nx, nt, T, u0, u1):
    dt = T / (nt - 1)
    dx = L / (nx - 1)

    r = np.sqrt(E * I / (rho * A)) * dt / dx**2

    U = np.zeros((nx, nt))

    # Apply initial conditions
    U[:, 0] = u0
    U[:, 1] = u0 + u1 * dt

    for j in range(1, nt - 1):
        for i in range(nx):
            
            # Apply boundary conditions
            if (i == 0): # At node 1
                if (BC1 == 3): # Free end
                    U[i][j+1] = 2*U[i][j] - U[i][j-1] - r**2 * (2*U[i][j] - 4*U[i+1][j] + 2*U[i+2][j])
            
            elif (i == 1): # At node 2
                if (BC1 == 1): # Fixed end
                    U[i][j+1] = 2*U[i][j] - U[i][j-1] - r**2 * (-4*U[i-1][j] + 7*U[i][j] - 4*U[i+1][j] + U[i+2][j])
                
                elif (BC1 == 2 or BC1 == 3): # Pinned end or Free end
                    U[i][j+1] = 2*U[i][j] - U[i][j-1] - r**2 * (-2*U[i-1][j] + 5*U[i][j] - 4*U[i+1][j] + U[i+2][j])

            elif (i == nx - 1): # At node nx
                if (BC2 == 3): # Free end
                    U[i][j+1] = 2*U[i][j] - U[i][j-1] - r**2 * (2*U[i-2][j] - 4*U[i-1][j] + 2*U[i][j])

            elif (i == nx - 2): # At node nx - 1
                if (BC2 == 1): # Fixed end
                    U[i][j+1] = 2*U[i][j] - U[i][j-1] - r**2 * (U[i+2][j] - 4*U[i-1][j] + 7*U[i][j] - 4*U[i+1][j])
                
                elif (BC2 == 2 or BC2 == 3): # Pinned end or Free end
                    U[i][j+1] = 2*U[i][j] - U[i][j-1] - r**2 * (U[i-2][j] - 4*U[i-1][j] + 5*U[i][j] - 2*U[i+1][j])
            
            # FD Scheme
            else:
                U[i][j+1] = 2*U[i][j] - U[i][j-1] - r**2 * (U[i-2][j] - 4*U[i-1][j] + 6*U[i][j] - 4*U[i+1][j] + U[i+2][j])

    return U

Numerical example:

Simply-supported beam

$$ u(x, t) = (\cos{\omega t} + \frac{1}{\omega}\sin{\omega t})\sin{\frac{\pi x}{L}}$$
where $ \omega = \pi^2 \sqrt{\frac{EI}{\rho A L^4}} $

In [ ]:
# Theoretical solution of transverse vibration of simply-supported beam 

def vibration_response_theoretical(E, I, A, L, rho, nx, nt, T):
    omega = (np.pi ** 2) * np.sqrt(E * I/ (rho * A * (L**4)))
    dt = T / (nt - 1)
    dx = L / (nx - 1)

    X_list = np.linspace(0, L, nx)
    t_list = np.linspace(0, T, nt)

    U_theoretical = np.zeros((nx, nt))

    for j in range(nt):
        for i in range(nx):
            U_theoretical[i][j] = (np.cos(omega * t_list[j]) + (1 / omega) * np.sin(omega * t_list[j])) * np.sin((np.pi / L) * X_list[i])


    return U_theoretical

In [ ]:
# Numerical data
E = I = A = rho = L = 1
T = 1
nx = 10
nt = 500

# Boundary conditions
BC1 = BC2 = 2


X_list = np.linspace(0, L, nx)
t_list = np.linspace(0, T, nt)

# Initial condition
u0 = np.sin((np.pi / L) * X_list)

In [ ]:
# Theoretical vibration response
U_th = vibration_response_theoretical(E, I, A, L, rho, nx, nt, T)

# Plot theoretical response
fig1, ax1 = plt.subplots(1,1)
cp1 = ax1.contourf(t_list, X_list, U_th, 100, cmap = 'jet')
fig1.colorbar(cp1) # Add a colorbar to a plot
ax1.set_title('Vibration Response Contour (Theoretical)')
ax1.set_xlabel('t (s)')
ax1.set_ylabel('x (m)')
plt.show()
#plt.savefig("plots/cont_th.svg")

In [ ]:
# Approximate vibration response using FDM
U_approx = transverse_vibration_response(E, I, A, rho, L, BC1, BC2, nx, nt, T, u0, u0)

# Plot approximate response
fig2, ax2 = plt.subplots(1,1)
cp2 = ax2.contourf(t_list, X_list, U_approx, 100, cmap = 'jet')
fig2.colorbar(cp2) # Add a colorbar to a plot
ax2.set_title('Vibration Response Contour (FDM)')
ax2.set_xlabel('t (s)')
ax2.set_ylabel('x (m)')
plt.show()
#plt.savefig("plots/cont_fdm_unstable.svg")

In [ ]:
# Comparison of first nx natural frequencies

omega_approx, phi_approx = transverse_vibration_modal(E, I, A, rho, L, BC1, BC2, nx)

omega_th = [i**2 * (np.pi ** 2 * np.sqrt(E * I / (rho * A * L**4))) for i in range(1, nx-1)]

fig_omega, ax_omega = plt.subplots(1,1)
ax_omega.plot([i for i in range(1, nx-1)], omega_th, marker = '.', label = "Theoretical")
ax_omega.plot([i for i in range(1, nx-1)], omega_approx, marker = '.', ls = '--', label = "FDM")
ax_omega.set_title("Natural Frequencies of Vibration")
ax_omega.set_xlabel("Mode")
ax_omega.set_ylabel("Frequency (rad/s)")
ax_omega.set_ylim(ymin = 0)
ax_omega.legend()
plt.show()
#plt.savefig("plots/freq.svg")

In [ ]:
# Plot mode shapes
fig_mode = plt.figure(figsize = (5, 6), constrained_layout = True)
ax_mode = fig_mode.subplots(nx-2, 1, sharex = True, sharey = True)
for i in range(nx-2):
    ax_mode[i].plot([i for i in range(1, nx + 1)], phi_approx[:, i], label = f"Mode-{i+1}", color = 'b')
    ax_mode[i].set_xlim([1, nx])
    ax_mode[i].legend(loc = "upper right")
    ax_mode[i].grid(True)
ax_mode[nx-3].set_xlabel("Node")
ax_mode[0].set_title("Mode Shapes of Vibration")
plt.show()
#plt.savefig("plots/modes.svg")

In [ ]:
# Root-mean-square error

Err = abs(U_th - U_approx)
rms = np.sqrt(np.sum(np.square(Err)) / (nx * nt))
print("% RMS Error =", rms * 100)

In [ ]:

# Animating the vibrating beam

import matplotlib.animation as animation

fig3, ax3 = plt.subplots()

def animate(i):
    ax3.clear()
    line1, = ax3.plot(X_list, U_th[:, i], color = 'red', linewidth = 2, label = "Analytical")
    line2, = ax3.plot(X_list, U_approx[:, i], color = 'blue', linewidth = 2, label = "Approximate (FDM)")
    ax3.grid(True)
    ax3.set_ylim([-2, 2])
    ax3.set_xlim([0, L])
    ax3.set_xlabel("x (m)")
    ax3.set_ylabel("Displacement (m)")
    ax3.set_title("Transverse Vibration of Beam Animation")
    ax3.legend()
    return line1, line2
    
anim = animation.FuncAnimation(fig3, animate, frames = nt, interval = 20)
#plt.show()
#plt.savefig("plots/beam_vib.svg")
from IPython.display import HTML
HTML(anim.to_jshtml())
